# Approximated Closeness Centrality

Libraries import & graph retrieval

In [3]:
import pickle               # to read the Amazon DiGraph
import networkx as nx       # to work with graphs

from datasketch import HyperLogLog
import copy
import time

In [4]:
graph_path = "../data/processed/amazon_graph.pickle"

with open(graph_path, "rb") as f:
    G = pickle.load(f)

## Hyperball Algorithm

HyperBall è l'algoritmo che usa i contatori HLL per calcolare la centralità su tutto il grafo in modo efficiente. L'idea geniale è che non serve fare una visita (BFS) per ogni nodo. Si possono "muovere" i contatori.

Il Funzionamento Passo-Passo:
1. Inizializzazione ($t=0$): Ogni nodo ha un contatore HLL che contiene solo se stesso. Quindi ogni nodo sa di poter raggiungere 1 persona (se stesso) a distanza 0.
2. Iterazione ($t=1, 2, \dots$): Immagina che ogni nodo dica ai suoi vicini: "Ehi, questi sono tutti i nodi che io so raggiungere". In termini tecnici: il contatore HLL di un nodo $x$ viene aggiornato facendone l'unione con i contatori HLL dei suoi vicini. Grazie alla matematica degli HLL, l'unione è un'operazione velocissima (massimizzazione dei registri).
3. Stima delle Distanze: Al passo $t$, il contatore del nodo $x$ stima quanti nodi sono raggiungibili in $\le t$ passi (chiamiamo questa stima $N_t$). Al passo precedente ($t-1$), sapevi quanti erano raggiungibili in $\le t-1$ passi ($N_{t-1}$). La differenza $N_t - N_{t-1}$ ti dice approssimativamente quanti nodi sono esattamente a distanza $t$.
4. Calcolo della Centralità: Sapendo quanti nodi sono a distanza $t$, puoi aggiungere quel numero moltiplicato per $\frac{1}{t}$ al totale della tua Harmonic Centrality.

In [5]:
def hyperball_harmonic_centrality(G, p=10):
    """
    Calcola la Harmonic Centrality approssimata usando HyperBall.

    Args:
        G: Oggetto NetworkX DiGraph (350k nodi).
        p: Precisione dei registri (p=10 -> 2^10 registri = errore ~3%).
           Valori più bassi usano meno RAM ma sono meno precisi.
    """

    # =========================================================================
    # FASE 1: PREPARAZIONE E INIZIALIZZAZIONE
    # =========================================================================
    # Per calcolare la centralità "in entrata" (quanto sono importante),
    # dobbiamo contare chi può raggiungere ME. HyperBall propaga "in avanti",
    # quindi lavoriamo sul grafo trasposto (archi invertiti).
    print(f"--- FASE 1: Inversione grafo e Inizializzazione HLL (p={p}) ---")
    G_rev = G.reverse()
    nodes = list(G_rev.nodes())

    # Dizionario per i contatori attuali: {nodo: HyperLogLog}
    # Inizialmente ogni nodo conosce solo se stesso (distanza 0).
    counters = {}
    for node in nodes:
        hll = HyperLogLog(p=p)
        # HLL richiede input in bytes. Convertiamo l'ID del nodo.
        node_id_encoded = str(node).encode('utf-8')
        # Aggiungo il nodo stesso al contatore (inizialmente l'insieme dei nodi raggiungibili contiene solo se stesso
        hll.update(node_id_encoded)
        # Aggiungo il contatore al dizionario che associa ogni nodo ad un contatore HyperLogLog
        counters[node] = hll

    # Dizionari per memorizzare i risultati
    # per ogni nodo in nodes, aggiungi al dizionario la coppia chiave-valore (node:0.0)
    harmonic_centrality = {node: 0.0 for node in nodes}

    # Memorizziamo la cardinalità al passo precedente (N_{t-1}).
    # Al tempo t=0, ogni nodo raggiunge solo se stesso, quindi count = 1.
    prev_cardinality = {node: 1.0 for node in nodes}

    print("Inizializzazione completata.")

    # =========================================================================
    # FASE 2: LOOP PRINCIPALE (Espansione della 'Palla') [cite: 771, 778]
    # =========================================================================
    # Iteriamo per t = 1, 2, ... fino a che le stime non cambiano più (stabilizzazione).
    # t rappresenta la distanza (raggiungibili in t passi).
    t = 0
    changed = True

    while changed:
        t += 1
        changed = False
        print(f"--- Inizio Iterazione t={t} ---")
        start_time = time.time()

        # Buffer per i nuovi contatori del passo t
        next_counters = {}

        # =====================================================================
        # FASE 3: UNIONE DEI CONTATORI (Propagazione) [cite: 775, 780]
        # =====================================================================
        # Logica: I nodi che posso raggiungere in t passi sono l'unione di:
        # 1. Quelli che raggiungevo già (me stesso e i vecchi percorsi)
        # 2. Quelli che raggiungono i miei vicini al passo t-1.

        for node in nodes:
            # Copiamo il contatore attuale del nodo (stato t-1)
            # NOTA: copy è necessario perché HLL è mutabile
            hll_new = copy.copy(counters[node])

            # Uniamo con i contatori dei vicini (successori nel grafo trasposto, G_rev.neighbors() = G.successors())
            # Questo simula il passaggio di informazioni "indietro" nel grafo originale
            neighbors = list(G_rev.neighbors(node))

            if neighbors:
                for neighbor in neighbors:
                    # L'operazione di unione HLL è molto veloce (bit-wise OR dei registri)
                    hll_new.merge(counters[neighbor])

            # Salviamo il nuovo stato
            next_counters[node] = hll_new

        # =====================================================================
        # FASE 4: AGGIORNAMENTO METRICA (Calcolo Harmonic)
        # =====================================================================
        # Formula: H(x) = sum [ (N_t - N_{t-1}) / t ]
        # Dove (N_t - N_{t-1}) è la stima dei nodi trovati ESATTAMENTE a distanza t.

        current_change_count = 0

        for node in nodes:
            old_count = prev_cardinality[node]
            new_count = next_counters[node].count()

            # Se la stima è aumentata, abbiamo trovato nuovi nodi a distanza t
            if new_count > old_count:
                delta = new_count - old_count

                # Aggiungiamo il contributo alla centralità armonica
                harmonic_centrality[node] += delta * (1.0 / t)

                # Aggiorniamo la memoria per il prossimo passo
                prev_cardinality[node] = new_count

                # Segnaliamo che c'è stato un cambiamento nel sistema
                changed = True
                current_change_count += 1

        # =====================================================================
        # FASE 5: CHIUSURA ITERAZIONE E CONTROLLO CONVERGENZA [cite: 833]
        # =====================================================================
        # Scambiamo i buffer: i contatori 'next' diventano quelli attuali per il prossimo t
        counters = next_counters

        elapsed = time.time() - start_time
        print(f"Fine t={t}. Nodi aggiornati: {current_change_count}. Tempo: {elapsed:.2f}s")

        # Sicurezza per evitare loop infiniti in grafi patologici,
        # anche se HLL converge tipicamente entro il diametro effettivo del grafo.
        if t > 1000: # Cutoff arbitrario, aumentabile
            print("Raggiunto limite massimo iterazioni.")
            break

    return harmonic_centrality

In [6]:
# ESEMPIO DI UTILIZZO
if __name__ == "__main__":
    # Calcolo
    centrality_scores = hyperball_harmonic_centrality(G, p=10)

    # Stampa top 5
    top_5 = sorted(centrality_scores.items(), key=lambda x: x[1], reverse=True)[:5]
    print("Top 5 nodi per Harmonic Centrality:", top_5)

--- FASE 1: Inversione grafo e Inizializzazione HLL (p=10) ---
Inizializzazione completata.
--- Inizio Iterazione t=1 ---
Fine t=1. Nodi aggiornati: 334843. Tempo: 43.41s
--- Inizio Iterazione t=2 ---
Fine t=2. Nodi aggiornati: 177205. Tempo: 45.98s
--- Inizio Iterazione t=3 ---


C:\Users\leona\AppData\Local\Temp\ipykernel_29544\1363427800.py:94: UserWarning: Warning: estimate is close to error correction threshold. Output may not satisfy HyperLogLog accuracy guarantee.
  new_count = next_counters[node].count()


Fine t=3. Nodi aggiornati: 151795. Tempo: 32.42s
--- Inizio Iterazione t=4 ---
Fine t=4. Nodi aggiornati: 135347. Tempo: 32.58s
--- Inizio Iterazione t=5 ---
Fine t=5. Nodi aggiornati: 123881. Tempo: 85.41s
--- Inizio Iterazione t=6 ---
Fine t=6. Nodi aggiornati: 115746. Tempo: 59.30s
--- Inizio Iterazione t=7 ---
Fine t=7. Nodi aggiornati: 109999. Tempo: 58.46s
--- Inizio Iterazione t=8 ---
Fine t=8. Nodi aggiornati: 105914. Tempo: 32.52s
--- Inizio Iterazione t=9 ---
Fine t=9. Nodi aggiornati: 102936. Tempo: 41.83s
--- Inizio Iterazione t=10 ---
Fine t=10. Nodi aggiornati: 100716. Tempo: 44.86s
--- Inizio Iterazione t=11 ---
Fine t=11. Nodi aggiornati: 98961. Tempo: 46.77s
--- Inizio Iterazione t=12 ---
Fine t=12. Nodi aggiornati: 97624. Tempo: 36.14s
--- Inizio Iterazione t=13 ---
Fine t=13. Nodi aggiornati: 96618. Tempo: 71.43s
--- Inizio Iterazione t=14 ---
Fine t=14. Nodi aggiornati: 95829. Tempo: 32.40s
--- Inizio Iterazione t=15 ---
Fine t=15. Nodi aggiornati: 95224. Tempo: 59.

In [7]:
import csv

def save_centrality_to_csv(centrality_dict, filename="harmonic_centrality_results.csv"):
    """
    Salva il dizionario delle centralità in un file CSV, ordinato per importanza decrescente.

    Args:
        centrality_dict (dict): Dizionario {nodo: score}
        filename (str): Nome del file di output
    """
    print(f"Avvio salvataggio su file: {filename}...")

    # 1. Ordinamento dei dati
    # Trasforma il dizionario in una lista di tuple (nodo, score) ordinata per score decrescente
    # key=lambda x: x[1] indica di ordinare in base al valore (lo score)
    sorted_data = sorted(centrality_dict.items(), key=lambda item: item[1], reverse=True)

    # 2. Scrittura su file
    try:
        # newline='' è importante su Windows per evitare righe vuote extra
        with open(filename, mode='w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)

            # Scrittura dell'intestazione (Header)
            writer.writerow(['ASIN', 'Harmonic_Centrality'])

            # Scrittura di tutte le righe in un colpo solo
            writer.writerows(sorted_data)

        print(f"Salvataggio completato con successo! ({len(sorted_data)} righe scritte)")
        print(f"Il nodo più centrale è '{sorted_data[0][0]}' con score {sorted_data[0][1]:.4f}")

    except IOError as e:
        print(f"Errore durante il salvataggio del file: {e}")

In [8]:
save_centrality_to_csv(centrality_scores, "../data/processed/harmonic_scores.csv")

Avvio salvataggio su file: harmonic_scores.csv...
Salvataggio completato con successo! (334843 righe scritte)
Il nodo più centrale è '0316769487' con score 18727.5339
